### Read the data from the input Excel file

In [5]:
import pandas as pd
inputFileName = "MineScheduling_miniDataSet.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
jobDF = pd.read_excel(inputFileName, "Job", skiprows=0)
jobMachineDF = pd.read_excel(inputFileName, "JobMachine", skiprows=0)
switchingDF = pd.read_excel(inputFileName, "Switching", skiprows=0)

### Set the data into dict + compute maxDuration

In [7]:
horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]


### Create your linear program

In [93]:
import pulp
prob = LpProblem("MineSchedulingProblem", LpMaximize)  

### Add variables

In [94]:
 #allocationVar[vJob][vMachine] & isFirst[vJob][vMachine]
allocationVar = dict()
isFirstVar = dict()
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in allocationVar : allocationVar[vJob] = dict() 
    if not vJob in isFirstVar : isFirstVar[vJob] = dict() 
    allocationVar[vJob][vMachine] = LpVariable("Allocation[%s][%s]"%(vJob,vMachine), cat='Binary')
    isFirstVar[vJob][vMachine] = LpVariable("isFirst[%s][%s]"%(vMachine, vJob), cat='Binary') 
    
#startVar[vJob] and isOver[vJob]
startVar = dict()
isOverVar = dict()
for vJob in jobDF['Id'] :
    startVar[vJob] = LpVariable("Start[%s]"%(vJob), lowBound=0, upBound=horizonDuration, cat='Continuous')
    isOverVar[vJob] = LpVariable("IsOver[%s]"%(vJob), lowBound=0, cat='Binary')

    
#isSuccessor[vMachine][vJob1-vJob2] = 1 if vJob2 is the direct successor of vJob1
isSuccessorVar = dict()  
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    vJob1 = switchingDF['Job1'][iMachine]
    vJob2 = switchingDF['Job2'][iMachine]
    if not vMachine in isSuccessorVar : isSuccessorVar[vMachine] = dict()
    if not vJob1 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob1] = dict() 
    if not vJob2 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob2] = dict() 
    isSuccessorVar[vMachine][vJob1][vJob2] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob1, vJob2), cat='Binary')
    isSuccessorVar[vMachine][vJob2][vJob1] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob2, vJob1), cat='Binary')  
                             

### Add objective

In [95]:

Production=jobDF.set_index('Id').to_dict()['Production']
Production
obj = lpSum([Production[i]*isOverVar[i] for i in Production.keys()])

#obj += lpSum(...)

prob += obj


### Add constraints

In [96]:
#Max one allocation per job
for vJob in jobDF['Id']:
    prob += lpSum([allocationVar[vJob][vMachine] for vMachine in allocationVar[vJob].keys()])<=1


In [97]:
#Max one "isFirst" job per
for vMachine in paramDF['Machine']:
        vJobs=jobMachineDF.loc[jobMachineDF['Machine'] == vMachine , 'Job']
        prob += lpSum([isFirstVar[vJob][vMachine] for vJob in vJobs])<=1

In [98]:
#Constraint stating the "start" variable
#Start[j’] >= Start[j] + duration[j][m] + switchingTime[m][j][j’]  - BigM * (1 - isSuccessor[m][j][j’])
#Start[j'] - Start[j] - BigM * isSuccessor[m][j][j'] >= duration[j][m] + switchingTime[m][j][j’] - BigM
BigM=100000
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    Job1=str(switchingDF['Job1'][iMachine])
    Job2=str(switchingDF['Job2'][iMachine])
    duration = float(jobMachineDF.loc[(jobMachineDF['Machine']==vMachine)&(jobMachineDF['Job']==Job1),'Duration'])
    SwitchingTime = float(switchingDF['SwitchingTime (day)'][iMachine])
    #print(Job1)
    #print(Job2)
    #print(duration)
    #print(SwitchingTime)
    prob+=startVar[Job2]>=startVar[Job1] + duration + SwitchingTime-BigM*(1-isSuccessorVar[vMachine][Job1][Job2])

In [103]:
#Constraint stating the "isOver" variable
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] <= durationHorizon + BigM * (1 - isOver[j])
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] + BigM * isOver[j] <= durationHorizon + BigM

for vJob in jobDF['Id']:
    duration = jobMachineDF[jobMachineDF['Job']==vJob]
    durationdict=dict(zip(duration.Machine, duration.Duration))
    prob+=startVar[vJob]+lpSum([durationdict[vMachine]*allocationVar[vJob][vMachine] for vMachine in durationdict.keys()])<=horizonDuration+BigM*(1-isOverVar[vJob])

In [17]:
#Constraint stating that to be allocated, a job shall either be the first, or be the successor of another job
# allocationVar[vJob][vMachine] <= isFirst[vJob][vMachine] + sum[otherJobs oJob] isSuccessor[vMachine][oJob][vJob]
for iJob,vJob in enumerate(jobMachineDF['Job']):
    allocationVar[vJob][vMachine]<=

In [18]:
#Constraint stating that if isOver = 1, then at least one allocation = 1
for vJob in jobDF['Id']:
    #XXX

In [19]:
#Max one successor per job
for iJob,vJob in enumerate(jobMachineDF['Job']):
    #XXX

### Solve and display results

In [106]:
prob.writeLP("mineSchedulingProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "IsOver" in v.name or "Start" in v.name or "Allocation" in v.name:
        if v.varValue != 0.0 : print(v.name, "=", v.varValue)

Status: Optimal
Objective =  474.0
IsOver_Job10_ = 1.0
IsOver_Job1_ = 1.0
IsOver_Job2_ = 1.0
IsOver_Job3_ = 1.0
IsOver_Job4_ = 1.0
IsOver_Job5_ = 1.0
IsOver_Job6_ = 1.0
IsOver_Job7_ = 1.0
IsOver_Job8_ = 1.0
IsOver_Job9_ = 1.0
